In [13]:
import os
import re
import numpy as np 
import time
import matplotlib.pyplot as plt 

In [14]:
def ext_data(direccion):
    with open(str(direccion), 'r') as archivo:
        nomb = ""
        dim = 0
        veh = 0
        cap = 0
        autos = []
        x_coord = []
        y_coord = []
        dema = {}  
        coordenadas = False  
        demandas = False  

        for linea in archivo:

            if linea.startswith("NAME"):
                nomb = (linea.strip().split(":")[1]).strip() + ".vrp"

            if linea.startswith("COMMENT"):
                resultado = re.search(r'of trucks:\s*(\d+)', linea, re.IGNORECASE)
                if resultado:
                    veh = int(resultado.group(1))

            if linea.startswith("VEHIC"):
                veh = int(linea.strip().split(":")[1])
                
            if linea.startswith("DIMEN"):
                dim = int(linea.strip().split(":")[1])

            elif linea.startswith("CAPAC"):
                cap = int(linea.strip().split(":")[1])

            elif linea.startswith("NODE_COORD_SECTION"):
                coordenadas = True
                continue

            elif linea.startswith("DEMAND_SECTION"):
                coordenadas = False
                demandas = True
                continue

            elif linea.startswith("DEPOT_SECTION"):
                demandas = False

            elif coordenadas:
                coords = linea.split()
                if len(coords) >= 3:  
                    x_coord.append(float(coords[1]))
                    y_coord.append(float(coords[2]))

            elif demandas:
                demands = linea.split()
                if len(demands) >= 2:
                    nodo, dem = int(demands[0]), int(demands[1])
                    dema[nodo - 1] = dem  

    for i in range(veh):
        autos.append(i+1)
        
    cap_x_veh = {i: cap for i in range(1, veh + 1)}

    return nomb, dim, cap_x_veh, x_coord, y_coord, dema, autos

In [15]:
#Funcion de Nearest_Neaighbor
def Nearest_Neaighbor(starting_node, clientes, distancia):
    NN=[starting_node]
    n = len(clientes)

    while len(NN) < n:
        k = NN[-1]
        nn = {(k,j): distancia[(k,j)] for j in clientes if j != k and j not in NN}
        nn.items()
        new = min(nn.items(), key = lambda x: x [1])
        NN.append(new[0][1])

  # NN.append(starting_node) #se elimina para no volver al nodo de inicio OVRP
    starting_node=0
    NN = Nearest_Neaighbor(starting_node, clientes, distancia)
    return NN


In [16]:
def listar_contenido_directorio(ruta_dic):
    try:
        
        contenido = os.listdir(ruta_dic)
        for elemento in contenido:
            nomb, dim, cap_x_veh, x_coord, y_coord, dema, autos = ext_data((str(ruta_dic) + "/" + str(elemento)))
            inst = nomb

            inicio = time.time()

            n = dim

            clientes = [i for i in range(n) if i != 0]

            nodos = [0] + clientes

            arcos = [(i,j) for i in nodos for j in nodos if i != j]

            q = dema

            vehiculos = autos

            Q = cap_x_veh

            X = x_coord
            Y = y_coord

            distancia = {(i,j): np.hypot(X[i] - X[j], Y[i] - Y[j]) for i in nodos for j in nodos if i != j}
            
            solucion = Nearest_Neaighbor(clientes, distancia)

            distancia_total = 0

            for vehiculo, ruta in solucion.items():
                distancia_recorrida_vehiculo = sum(distancia[(ruta[i], ruta[i+1])] for i in range(len(ruta)-1))
                
                distancia_total += distancia_recorrida_vehiculo

            fin = time.time()
            tiempo_transcurrido = fin - inicio
            print("---------------------------------")
            print("\n")
            print("El script tardó {:.5f} segundos en ejecutar".format(tiempo_transcurrido), "la instancia", inst)
            print("Distancia total recorrida:", distancia_total)
            print("\n")
    except PermissionError:
        print(f"No se tienen permisos para acceder al directorio {ruta_dic}")

ruta_directorio = os.getcwd()

for raiz, directorios, archivos in os.walk(ruta_directorio):
    for nombre_directorio in directorios:
        ruta_completa = os.path.join(raiz, nombre_directorio)
        listar_contenido_directorio(ruta_completa)

No se tienen permisos para acceder al directorio c:\Users\diego\OneDrive\Escritorio\Final_Tesis\Codigo\INSTANCES


TypeError: Nearest_Neaighbor() missing 1 required positional argument: 'distancia'